In [45]:
import requests
import os

STOCK = "TSLA"
COMPANY_NAME = "Tesla Inc"

url = 'https://www.alphavantage.co/query'
api_key = os.environ.get("STOCK_API_KEY")
params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": STOCK,
    "apikey": api_key,
}
response = requests.get(url, params=params)
data = response.json()

In [32]:
data

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'TSLA',
  '3. Last Refreshed': '2023-11-03',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2023-11-03': {'1. open': '221.1500',
   '2. high': '226.3701',
   '3. low': '218.4000',
   '4. close': '219.9600',
   '5. volume': '119534790'},
  '2023-11-02': {'1. open': '212.9700',
   '2. high': '219.2000',
   '3. low': '211.4500',
   '4. close': '218.5100',
   '5. volume': '125987621'},
  '2023-11-01': {'1. open': '204.0400',
   '2. high': '205.9900',
   '3. low': '197.8500',
   '4. close': '205.6600',
   '5. volume': '121661656'},
  '2023-10-31': {'1. open': '196.1200',
   '2. high': '202.8000',
   '3. low': '194.0700',
   '4. close': '200.8400',
   '5. volume': '118068273'},
  '2023-10-30': {'1. open': '209.2800',
   '2. high': '210.8800',
   '3. low': '194.6700',
   '4. close': '197.3600',
   '5. volume': '136448167'},
  '2023-10-27': {'1. open':

In [33]:
from datetime import date
from datetime import timedelta
 
# Get today's date
today = date.today()
print("Today is: ", today)
 
# Yesterday date
yesterday = today - timedelta(days = 1)
day_before_yesterday = today - timedelta(days = 2)
print("Yesterday was: ", yesterday)
print("2 days ago was: ", day_before_yesterday)

Today is:  2023-11-04
Yesterday was:  2023-11-03
2 days ago was:  2023-11-02


In [34]:
yesterday_close_price = data["Time Series (Daily)"][str(yesterday)]['4. close']
# convert str to float
yesterday_close_price = float(yesterday_close_price)

two_days_ago_close_price = data["Time Series (Daily)"][str(day_before_yesterday)]['4. close']
# convert str to float
two_days_ago_close_price = float(two_days_ago_close_price)

print(f"Yesterday close price: {yesterday_close_price}")
print(f"2 Days ago close price: {two_days_ago_close_price}")

Yesterday close price: 219.96
2 Days ago close price: 218.51


In [35]:
stock_change = (yesterday_close_price - two_days_ago_close_price)/two_days_ago_close_price * 100

if stock_change > 0:
    emoji = "🔺"
elif stock_change < 0:
    emoji = "🔻"
    
print(f"stock change percentage is: {stock_change}")

stock change percentage is: 0.6635851906091333


In [36]:
def get_news():
    news_api_url = "https://newsapi.org/v2/everything"
    news_api = os.environ.get("NEWS_API")
    
    news_params = {
        "q": COMPANY_NAME,
        "from": day_before_yesterday,
        "to": yesterday,
        "sortBy": "popularity",
        "apiKey": news_api,
    }
    
    all_articles = requests.get(url=news_api_url, params=news_params)
    all_articles.raise_for_status()
    all_articles = all_articles.json()
    # get first 3 articles description and save in a list
    articles = all_articles["articles"][:3]
    articles_title = [article["title"] for article in articles]
    articles_description = [article["description"] for article in articles]
    return articles_title, articles_description

In [37]:
def send_sms(articles_title, articles_description):
    from twilio.rest import Client
    import os
    
    account_sid = os.environ.get("TWILIO_ACCOUNT_SID")
    auth_token = os.environ.get("TWILIO_AUTH_TOKEN")
    
    for _ in range(3):
        client = Client(account_sid, auth_token)
        message = client.messages \
            .create(
            body=f"{STOCK} {stock_change}% {emoji}\nHeading: {articles_title[_]}\n Brief: {articles_description[_]}",
            from_="TWILIO_PHONE_NUMBER",
            to="PHONE_NUMBER_FOR_NEWS_ALERT"
        )
    print(message.status)

In [43]:
if stock_change > 5 or stock_change < -5:
    articles_title, articles_description = get_news()
    send_sms(articles_title, articles_description)

queued
